In [14]:
#  imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [15]:
# load data
from keras.datasets import cifar10

cls_names = ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

"""
X_train.shape: (50000, 3, 32, 32)
X_test.shape: (10000, 3, 32, 32)
y: 10 labels
"""
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = y_train.reshape((y_train.shape[0]))
y_test = y_test.reshape((y_test.shape[0]))

In [16]:
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

In [17]:
X_train.shape

(50000, 3072)

In [18]:
# Deep RandomForest
import argparse
import numpy as np
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/cifar10.json")  
gc = GCForest(config)

In [19]:
# count class
len(np.unique(y_train))

10

In [23]:
# X_enc is the concatenated predict_proba result of RandomForest

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    

[ 2018-04-21 23:58:19,110][cascade_classifier.fit_transform] X_groups_train.shape=[(50000, 3072)],y_train.shape=(50000,),X_groups_test.shape=[(10000, 3072)],y_test.shape=(10000,)
[ 2018-04-21 23:58:19,244][cascade_classifier.fit_transform] group_dims=[3072]
[ 2018-04-21 23:58:19,246][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-21 23:58:19,248][cascade_classifier.fit_transform] group_ends=[3072]
[ 2018-04-21 23:58:19,249][cascade_classifier.fit_transform] X_train.shape=(50000, 3072),X_test.shape=(10000, 3072)
[ 2018-04-21 23:58:19,494][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(50000, 3072), X_cur_test.shape=(10000, 3072)
[ 2018-04-21 23:58:26,997][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=46.00%
[ 2018-04-21 23:58:34,718][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=46.06%
[ 2018-04-21 23:58:42,320][kfold_wrapper.log_eval_metrics] Accuracy

In [24]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of RandomForest = {:.6f} %".format(acc * 100))

[ 2018-04-21 23:59:41,961][cascade_classifier.transform] X_groups_test.shape=[(10000, 3072)]
[ 2018-04-21 23:59:41,990][cascade_classifier.transform] group_dims=[3072]
[ 2018-04-21 23:59:41,993][cascade_classifier.transform] X_test.shape=(10000, 3072)
[ 2018-04-21 23:59:42,058][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(10000, 3072)


Test Accuracy of RandomForest = 49.150000 %


In [25]:
# predict_proba result of RandomForest + data 
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-21 23:59:46,165][cascade_classifier.transform] X_groups_test.shape=[(10000, 3072)]
[ 2018-04-21 23:59:46,174][cascade_classifier.transform] group_dims=[3072]
[ 2018-04-21 23:59:46,176][cascade_classifier.transform] X_test.shape=(10000, 3072)
[ 2018-04-21 23:59:46,206][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(10000, 3072)


In [26]:
X_test_enc.shape

(10000, 3082)

In [27]:
X_train_new = X_train_enc
X_test_new  = X_test_enc

In [28]:
X_train_new.shape

(50000, 3082)

In [29]:
# Cascade randomForest
config = load_json("./examples/cifar10gc.json")  
gc = GCForest(config)

X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-21 23:59:55,281][cascade_classifier.fit_transform] X_groups_train.shape=[(50000, 3082)],y_train.shape=(50000,),X_groups_test.shape=[(10000, 3082)],y_test.shape=(10000,)
[ 2018-04-21 23:59:55,566][cascade_classifier.fit_transform] group_dims=[3082]
[ 2018-04-21 23:59:55,569][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-21 23:59:55,570][cascade_classifier.fit_transform] group_ends=[3082]
[ 2018-04-21 23:59:55,572][cascade_classifier.fit_transform] X_train.shape=(50000, 3082),X_test.shape=(10000, 3082)
[ 2018-04-21 23:59:55,912][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(50000, 3082), X_cur_test.shape=(10000, 3082)
[ 2018-04-22 00:00:03,179][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=47.52%
[ 2018-04-22 00:00:10,969][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=46.62%
[ 2018-04-22 00:00:18,864][kfold_wrapper.log_eval_metrics] Accuracy

[ 2018-04-22 00:05:32,042][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=48.14%
[ 2018-04-22 00:05:39,712][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=49.20%
[ 2018-04-22 00:05:47,073][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=48.86%
[ 2018-04-22 00:05:55,041][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=47.66%
[ 2018-04-22 00:06:02,398][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=49.78%
[ 2018-04-22 00:06:10,253][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=48.88%
[ 2018-04-22 00:06:17,759][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=49.84%
[ 2018-04-22 00:06:25,343][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=49.32%


In [30]:
def update(X_train,X_test):
    # update Estimators
    for i in range(5):
        M = np.delete(X_train, (-1), axis=1)
        X_train= np.delete(M, (-1), axis=1)
        M = np.delete(X_test, (-1), axis=1)
        X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of RandomForest = {:.6f} %".format(acc * 100),i,'fois')

In [31]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    
    config = load_json("./examples/cifar10gc.json")  
    gc = GCForest(config)
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()    

[ 2018-04-22 00:10:36,065][cascade_classifier.fit_transform] X_groups_train.shape=[(50000, 3082)],y_train.shape=(50000,),X_groups_test.shape=[(10000, 3082)],y_test.shape=(10000,)
[ 2018-04-22 00:10:36,399][cascade_classifier.fit_transform] group_dims=[3082]
[ 2018-04-22 00:10:36,401][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-22 00:10:36,402][cascade_classifier.fit_transform] group_ends=[3082]
[ 2018-04-22 00:10:36,404][cascade_classifier.fit_transform] X_train.shape=(50000, 3082),X_test.shape=(10000, 3082)
[ 2018-04-22 00:10:36,736][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(50000, 3082), X_cur_test.shape=(10000, 3082)
[ 2018-04-22 00:10:44,415][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=48.02%
[ 2018-04-22 00:10:52,358][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=48.48%
[ 2018-04-22 00:11:00,770][kfold_wrapper.log_eval_metrics] Accuracy

[ 2018-04-22 00:16:07,796][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=49.30%
[ 2018-04-22 00:16:17,169][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=50.04%
[ 2018-04-22 00:16:27,172][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=49.92%
[ 2018-04-22 00:16:36,007][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=48.52%
[ 2018-04-22 00:16:45,239][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=48.66%
[ 2018-04-22 00:16:54,352][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=49.02%
[ 2018-04-22 00:17:02,706][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=49.52%
[ 2018-04-22 00:17:10,154][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=49.34%


('Test Accuracy of RandomForest = 51.290000 %', 0, 'fois')


[ 2018-04-22 00:21:39,960][cascade_classifier.fit_transform] X_groups_train.shape=[(50000, 3082)],y_train.shape=(50000,),X_groups_test.shape=[(10000, 3082)],y_test.shape=(10000,)
[ 2018-04-22 00:21:40,219][cascade_classifier.fit_transform] group_dims=[3082]
[ 2018-04-22 00:21:40,221][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-22 00:21:40,222][cascade_classifier.fit_transform] group_ends=[3082]
[ 2018-04-22 00:21:40,224][cascade_classifier.fit_transform] X_train.shape=(50000, 3082),X_test.shape=(10000, 3082)
[ 2018-04-22 00:21:40,500][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(50000, 3082), X_cur_test.shape=(10000, 3082)
[ 2018-04-22 00:21:48,671][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=47.64%
[ 2018-04-22 00:21:56,505][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=48.60%
[ 2018-04-22 00:22:04,072][kfold_wrapper.log_eval_metrics] Accuracy

[ 2018-04-22 00:27:17,734][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=48.08%
[ 2018-04-22 00:27:25,452][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=49.30%
[ 2018-04-22 00:27:32,932][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=49.12%
[ 2018-04-22 00:27:40,684][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=48.74%
[ 2018-04-22 00:27:48,512][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=48.96%
[ 2018-04-22 00:27:56,036][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=49.30%
[ 2018-04-22 00:28:03,680][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=49.88%
[ 2018-04-22 00:28:11,009][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=48.60%


...

('Test Accuracy of RandomForest = 51.490000 %', 18, 'fois')


[ 2018-04-22 04:00:13,278][cascade_classifier.fit_transform] X_groups_train.shape=[(50000, 3082)],y_train.shape=(50000,),X_groups_test.shape=[(10000, 3082)],y_test.shape=(10000,)
[ 2018-04-22 04:00:13,544][cascade_classifier.fit_transform] group_dims=[3082]
[ 2018-04-22 04:00:13,546][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-22 04:00:13,547][cascade_classifier.fit_transform] group_ends=[3082]
[ 2018-04-22 04:00:13,549][cascade_classifier.fit_transform] X_train.shape=(50000, 3082),X_test.shape=(10000, 3082)
[ 2018-04-22 04:00:13,815][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(50000, 3082), X_cur_test.shape=(10000, 3082)
[ 2018-04-22 04:00:21,213][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=48.04%
[ 2018-04-22 04:00:28,633][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=49.10%
[ 2018-04-22 04:00:35,854][kfold_wrapper.log_eval_metrics] Accuracy

[ 2018-04-22 04:05:37,430][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=49.32%
[ 2018-04-22 04:05:45,105][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=49.34%
[ 2018-04-22 04:05:52,478][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=50.34%
[ 2018-04-22 04:05:59,824][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=49.26%
[ 2018-04-22 04:06:07,496][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=49.14%
[ 2018-04-22 04:06:16,152][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=49.12%
[ 2018-04-22 04:06:23,503][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=49.90%
[ 2018-04-22 04:06:31,230][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=48.78%


[ 2018-04-22 04:11:31,913][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.train_7.predict)=49.64%
[ 2018-04-22 04:11:39,509][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.train_8.predict)=49.36%
[ 2018-04-22 04:11:46,900][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.train_9.predict)=50.26%
[ 2018-04-22 04:11:47,075][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.train_cv.predict)=49.40%
[ 2018-04-22 04:11:47,078][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.test.predict)=51.44%
[ 2018-04-22 04:11:47,090][cascade_classifier.calc_accuracy] Accuracy(layer_8 - train.classifier_average)=49.40%
[ 2018-04-22 04:11:47,093][cascade_classifier.calc_accuracy] Accuracy(layer_8 - test.classifier_average)=51.44%
[ 2018-04-22 04:11:47,095][cascade_classifier.fit_transform] [Result][Optimal Level Detected] opt_layer_num=6, accuracy_train=49.66%, accuracy_test=51.5

('Test Accuracy of RandomForest = 51.530000 %', 19, 'fois')


# ('Test Accuracy of RandomForest = 51.950000 %', 3, 'fois')